In [66]:
import scattertext as ST
import re
from pprint import pprint
import pandas as pd
import spacy.en
from IPython.display import IFrame

In [58]:
nlp = spacy.en.English()

In [9]:
lines = open('presidential-debate-2016-09-26.txt').read().split('\n')

In [53]:
cur_speaker = None
speaker_start_re = re.compile(r'^([(]?[A-Z][A-Z][A-Z]+):?(.+)$')
transcript = []
cur_statement = ''
cur_speaker = None
for line in lines:
    match = speaker_start_re.match(line)
    if match:
        if match.group(1).startswith('('):
            continue
        if cur_speaker is not None:
            transcript.append({'speaker': cur_speaker, 'statement': cur_statement})
        cur_speaker = match.group(1).strip()
        cur_statement = match.group(2).strip() + '\n'
        for other_name in ['TRUMP','CLINTON','HOLT']:
            if other_name+':' in cur_statement:
                cur_statement, other_statement = cur_statement.split(other_name)
                transcript.append({'speaker': cur_speaker, 'statement': cur_statement.strip()})
                transcript.append({'speaker': other_name, 'statement': other_statement.strip()})   
    else:
        cur_statement += line 
df = pd.DataFrame(transcript)


In [55]:
df.iloc[:4]

,speaker,statement
0,HOLT,Good evening from Hofstra University in Hempst...
1,CLINTON,"How are you, Donald?\n"
2,HOLT,"Good luck to you.\nWell, I don't expect us to ..."
3,CLINTON,"Well, thank you, Lester, and thanks to Hofstra..."


In [61]:
term_doc_mat = ST.TermDocMatrixFromPandas(data_frame = df[(df['speaker'] == 'CLINTON') | (df['speaker'] == 'TRUMP')], 
                                          category_col = 'speaker', 
                                          text_col = 'statement',
                                          nlp = nlp).build()

In [64]:
html = st.produce_scattertext_html(term_doc_mat, category='CLINTON', 
                                   category_name='Clinton', not_category_name='Trump')

In [68]:
open('20160926debate.html', 'wb').write(html.encode('utf-8'))
IFrame(src='20160926debate.html', width = 1000, height=1000)